In [2]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import balanced_accuracy_score, confusion_matrix
import pickle


In [3]:
def accuracy(X_test, y_test, model):
    y_pred = model.predict(X_test)
    M = confusion_matrix(y_test, y_pred)
    print(M)
    TPR = M[0, 0] / (M[0, 0] + M[0, 1])
    TNR = M[1, 1] / (M[1, 0] + M[1, 1])
    print(round(TPR, 2), round(TNR, 2), '\n')

In [4]:
df = pd.read_pickle("cc_data.pkl")
ann = pd.read_pickle("cc_ann.pkl")

In [31]:
X = df.loc[ann.index].to_numpy()
y = ann["Class"].to_numpy()


In [5]:
X_train = df.loc[ann.loc[ann["Dataset type"] == "Training"].index].to_numpy()
y_train = ann.loc[ann["Dataset type"] == "Training", "Class"].to_numpy()

X_test1 = df.loc[ann.loc[ann["Dataset"] == "GSE14333"].index].to_numpy()
y_test1 = ann.loc[ann["Dataset"] == "GSE14333", "Class"].to_numpy()
X_test2 = df.loc[ann.loc[ann["Dataset"] == "GSE37892"].index].to_numpy()
y_test2 = ann.loc[ann["Dataset"] == "GSE37892", "Class"].to_numpy()
X_test3 = df.loc[ann.loc[ann["Dataset"] == "GSE33113"].index].to_numpy()
y_test3 = ann.loc[ann["Dataset"] == "GSE33113", "Class"].to_numpy()

In [98]:
df = pd.read_pickle("cc_data.pkl").loc[:, ["FABP6", "AKAP12", "SLC9A3R1", "VEGFA"]]
df = pd.read_pickle("cc_data.pkl").loc[:, ["FAP", "INHBA", "BGN", "MKI67", "MYC", "MYBL2", "GADD45B", "ATP5F1E", "PGK1", "VDAC2"]]
df = pd.read_pickle("cc_data.pkl").loc[:, ["FABP6", "AKAP12", "SLC9A3R1", "VEGFA", "MYC"]]
df

# VEGFA ?? SLC9A3R1 ?? RHEB

,FAP,INHBA,BGN,MKI67,MYC,MYBL2,GADD45B,ATP5F1E,PGK1,VDAC2
GSM358386,8.217333,10.318609,8.511807,6.810508,10.857053,6.784541,8.186348,11.830243,12.121546,10.453021
GSM358387,7.677184,9.394449,8.161911,8.085656,9.697597,7.582425,7.777452,12.306194,12.639473,10.745512
GSM358388,9.214904,11.303818,9.778336,7.608049,10.910118,7.303816,8.331115,11.361403,12.182339,10.823994
GSM358389,8.152389,9.295833,7.543559,8.334858,9.982777,7.217153,7.395177,11.989095,11.834170,10.330193
GSM358394,8.457863,10.706797,9.167547,7.838564,10.870022,8.099214,8.601300,12.510738,12.189914,10.918167
...,...,...,...,...,...,...,...,...,...,...
GSM972518,7.443426,8.367780,9.175415,8.841427,11.196240,7.906850,8.655989,12.064314,12.594548,10.959504
GSM972519,7.479213,8.607823,9.309701,8.339628,11.949093,8.565875,8.975933,12.459628,12.149520,11.050865
GSM972520,7.352982,9.579617,6.675953,7.812581,10.965494,8.012959,8.285140,12.622328,12.205967,11.844019
GSM972521,8.552466,9.360301,9.190637,8.294099,12.395502,7.740024,8.685913,12.650404,12.550693,10.860852


In [40]:
genes = ['HOXA7', 'FABP6', 'GRIP2', 'VEGFA', 'AKAP12', 'RHEB', 'PMEPA1']
df.loc[:, genes]

,HOXA7,FABP6,GRIP2,VEGFA,AKAP12,RHEB,PMEPA1
GSM358386,7.560018,7.539222,6.305451,11.310581,9.971291,11.214446,10.884226
GSM358387,6.748290,8.181142,6.374782,9.799357,7.898547,10.659847,9.059112
GSM358388,7.046598,7.574915,6.700481,10.822698,9.254585,11.161679,9.657010
GSM358389,6.854425,6.726371,6.406703,9.638734,8.028811,10.898204,9.480929
GSM358394,7.342370,7.479381,6.317011,10.835058,9.944802,11.221793,11.211368
...,...,...,...,...,...,...,...
GSM972518,9.025089,8.744051,6.662329,9.832793,7.612523,11.447410,9.890117
GSM972519,8.373009,7.200391,6.417680,9.618347,7.214963,11.385949,11.201527
GSM972520,7.695044,8.516651,6.157838,10.839279,7.646596,11.763227,10.013880
GSM972521,8.475380,8.297815,6.166685,10.544763,9.398063,11.636443,10.970792


In [22]:
model2 = CatBoostClassifier(random_seed=17,
                            learning_rate=1,
                            verbose=True)

In [32]:
model2.fit(X_train, y_train, eval_set=(X_test1, y_test1))

0:	learn: 0.5542799	test: 0.6151741	best: 0.6151741 (0)	total: 10.4s	remaining: 2h 53m 56s
1:	learn: 0.4854989	test: 0.5022289	best: 0.5022289 (1)	total: 18.5s	remaining: 2h 34m 2s
2:	learn: 0.4308207	test: 0.4712612	best: 0.4712612 (2)	total: 21.3s	remaining: 1h 57m 48s
3:	learn: 0.3557276	test: 0.3155667	best: 0.3155667 (3)	total: 23.8s	remaining: 1h 38m 49s
4:	learn: 0.3270894	test: 0.2731428	best: 0.2731428 (4)	total: 26.4s	remaining: 1h 27m 41s
5:	learn: 0.2783389	test: 0.2617147	best: 0.2617147 (5)	total: 28.8s	remaining: 1h 19m 35s


KeyboardInterrupt: 

In [25]:
m = model2

print("Test batch 1"), accuracy(X_test1, y_test1, m)
print("Test batch 2"), accuracy(X_test2, y_test2, m)
print("Test batch 3"), accuracy(X_test3, y_test3, m)
print("Train batch"), accuracy(X_train, y_train, m)

Test batch 1
[[55  6]
 [33  6]]
0.9 0.15 

Test batch 2
[[39  9]
 [ 8 11]]
0.81 0.58 

Test batch 3
[[28  2]
 [14  3]]
0.93 0.18 

Train batch
[[169  33]
 [ 49  67]]
0.84 0.58 



(None, None)

In [10]:
y_test3

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1], dtype=int64)

In [ ]:
get_gpu_device_count()

In [ ]:
model.save_model("model1.cbm")

In [6]:
from_file = CatBoostClassifier()

from_file.load_model("model1.cbm")
